In [45]:
using ForwardDiff
using DifferentialEquations
const DE = DifferentialEquations
using Zygote
using DiffEqSensitivity
using PyPlot
using CUDA
using LinearAlgebra

In [75]:
function f(a)
    A = a*[.1 2; .3 .4]
    b = [1/sqrt(2); 1/sqrt(2)]
    for i=1:100
        b = A*b
    end
    return b[1]
end     
f_grad = x -> ForwardDiff.derivative(f,x)

#17 (generic function with 1 method)

In [138]:
function f(a)
    p0 = a
    f(u,p,t) = p*u
    u0 = 1
    for i=1:100
        tspan = ((i-1)/100,i/100)
        prob = DE.ODEProblem(f,u0,tspan, p=p0)
        tmp_prob = remake(prob, p=p0)
        sol = DE.solve(tmp_prob, Tsit5(); reltol=1e-8, abstol=1e-8)
        u0 = sol.u[end]
    end
    return u0
end
f_grad = x -> ForwardDiff.derivative(f,x)

#69 (generic function with 1 method)

In [139]:
@time f_grad(1.0) 

  1.227274 seconds (2.51 M allocations: 140.309 MiB, 4.15% gc time, 99.45% compilation time)


2.71828182845909

In [140]:
@time f_grad(1.0)

  0.006111 seconds (30.10 k allocations: 1.910 MiB)


2.71828182845909

In [213]:
function test3(a)
    p0 = a
    f(u,p,t) = p*u
    u0 = 0.0*im
    for i=1:10
        tspan = (0.0,2.0)
        prob = DE.ODEProblem(f,u0,tspan, p=p0)
        tmp_prob = remake(prob, p=p0)
        sol = DE.solve(tmp_prob, Tsit5(); reltol=1e-8, abstol=1e-8)
        u0 = sol.u[end]
    end
    return real(u0)
end

test3 (generic function with 1 method)

In [214]:
test3(1.0*im)

0.0

In [215]:
DF(test3) = x -> Zygote.gradient(test3, x)

DF (generic function with 1 method)

In [216]:
DF(test3)(1.0)

LoadError: MethodError: no method matching similar(::Float64, ::Int64)
[0mClosest candidates are:
[0m  similar([91m::PyCall.PyVector{T}[39m, ::Int64...) where T at ~/.julia/packages/PyCall/7a7w0/src/conversions.jl:268
[0m  similar([91m::PyCall.PyVector[39m, ::Any, [91m::Tuple{Vararg{Int64, N}} where N[39m) at ~/.julia/packages/PyCall/7a7w0/src/conversions.jl:265
[0m  similar([91m::ReverseDiff.TrackedArray[39m, ::Union{Integer, AbstractUnitRange}...) at ~/.julia/packages/ReverseDiff/Z4pL0/src/tracked.jl:387
[0m  ...

# Playground

In [6]:
f = x -> x^2

#6 (generic function with 1 method)

In [10]:
@time DF(f)(12)

  0.000012 seconds


24

test3 (generic function with 1 method)

In [159]:
test3_grad = x -> Zygote.gradient(test3,x);

In [160]:
test3([1.01])# + 0*im)

7.538324935121546